In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Dropout,Conv2DTranspose,Add,GlobalAveragePooling2D,Input,Conv2D,MaxPool2D,Concatenate,BatchNormalization,Activation
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10
import numpy as np
import os

Using TensorFlow backend.
/home/lalit/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lalit/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lalit/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lalit/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
(X_train,Y_train),(X_test,Y_test)=cifar10.load_data()

In [3]:
BATCH_SIZE = 32 
EPOCHS = 200
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
Y_train =to_categorical(Y_train, 10)
Y_test =to_categorical(Y_test, 10)

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [6]:

in_layer = Input(shape=[32, 32, 3])
conv_1=Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(in_layer)
pool1=MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv_1)
x=Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same')(pool1)
x =Conv2DTranspose(64,(4,4),padding='same',activation='relu',strides=2)(x)

for i in range(5):
  x=Conv2D(64, kernel_size=(3, 3), , padding='same')(x)

for i in range(8):
  x=Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)

for i in range(12):
  x=Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)

for i in range(6):
  x=Conv2D(512, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)

x=GlobalAveragePooling2D()(x)
#x= Flatten()(x)
x=Dense(10,activation='softmax',kernel_initializer='he_normal')(x)
model = Model(inputs=[in_layer], outputs=[x])
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=lr_schedule(0)),metrics=['accuracy'])

model.summary()

SyntaxError: invalid syntax (<ipython-input-6-9c053cf52848>, line 9)

In [ ]:
model.fit(X_train, Y_train,batch_size=BATCH_SIZE,epochs=EPOCHS,shuffle=True)

In [ ]:
datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)


datagen.fit(X_train)


model.fit_generator(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),validation_data=(X_test, Y_test),epochs=EPOCHS, verbose=0, workers=1,use_multiprocessing=False)